In [3]:
import tensorflow as tf
import numpy as np

In [189]:
ENV_SIZE = 16
INITIAL_POSITION = 1.0
FOOD_POSITION = 8.0
LEARNING_RATE = 0.01


tf.reset_default_graph()
Psi = tf.Variable(initial_value=INITIAL_POSITION, trainable=False, name='external')  # external states
S = tf.Variable(initial_value=1.0, trainable=False, name='sensory')  # sensory states
A = tf.Variable(initial_value=1.0, trainable=False, name='action')  # action states
R = tf.Variable(initial_value=[1/16]*16, trainable=True, name='internal')  # internal states


def environmental_dynamics(psi, a):
    # How position changes given action state
    return tf.cast(psi + a, tf.float32)


def sensory_dynamics(psi):
    # P(s | psi)
    # Sensory states given external states
    k = 4**(-1/16)
    omega = np.log(4)/16
    return k * tf.exp(-omega * tf.abs(psi- FOOD_POSITION))


def model_encoding(R):
    # P(psi | r)
    # External state given internal state
    return tf.nn.softmax(R)


def generative_density(R, a):
    return environmental_dynamics(sensory_dynamics(model_encoding(R)), a)


def variational_density(R):
    """
    Agent belief about the external states (i.e. its current position in the 
    world) as encoded in internal states
    """
    return tf.nn.softmax(R)


def KL(a, b):
    """
    Kullback-Leibler divergence between densities a and b
    """
    return tf.reduce_sum(
        a * tf.log(a/b)
    )


def free_energy(R, s, a):
    divergence = KL(variational_density(R), generative_density(R, a))
    surprisal = tf.log(tf.multiply(s, a))
    return divergence - surprisal

cost = free_energy(R, S, A)

In [192]:
# Initializing the variables
init = tf.global_variables_initializer()

# Initialize optimizer
optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cost)

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    for var in tf.global_variables():
        print('{name} = {value}'.format(name=var.name, value=var.eval(session=sess)))
    
    print(generative_density(R, A).eval())
    print(free_energy(R, S, A).eval())
    
    summary_writer = tf.summary.FileWriter('logs', graph=tf.get_default_graph())

    for epoch in range(15):
        sess.run(
            S.assign(
                np.random.choice(
                    [1, 0],
                    p=[sensory_dynamics(Psi).eval(), 1-sensory_dynamics(Psi).eval()])
            )
        )
        sess.run(A.assign(
                 [-1, 1][np.argmin([free_energy(R, S, -1).eval(), free_energy(R, S, 1).eval()])]
             )
        )
        sess.run(optimizer)    
        sess.run(Psi.assign(environmental_dynamics(Psi, A) % ENV_SIZE))
        
        print('cost = ', cost.eval())
        print('KL = ', KL(variational_density(R), generative_density(R, A)).eval())
        print('gen', generative_density(R, A).eval())
        print('gen', variational_density(R).eval())
        for var in tf.global_variables():
            print('{name} = {value}'.format(name=var.name, value=var.eval(session=sess)))

external:0 = 1.0
sensory:0 = 1.0
action:0 = 1.0
internal:0 = [ 0.0625  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625
  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625  0.0625]


[ 1.46099162  1.46099162  1.46099162  1.46099162  1.46099162  1.46099162
  1.46099162  1.46099162  1.46099162  1.46099162  1.46099162  1.46099162
  1.46099162  1.46099162  1.46099162  1.46099162]


-3.1517


cost =  nan


KL =  nan


...

gen [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan]
external:0 = 3.0
sensory:0 = 1.0
action:0 = -1.0
internal:0 = [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan]


cost =  nan


KL =  nan


gen [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan]


gen [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan]
external:0 = 2.0
sensory:0 = 0.0
action:0 = -1.0
internal:0 = [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan]
